Previous training with the initial 7 features gives high importance to the features mu, RA and t. So next step we can create new features using operations * and / with these features as component, that is, we create new features $RA\cdot t, mu/RA, mu/t$, add them into the training set and retrain the model.

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.inspection import permutation_importance
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import Callback

In [3]:
# Load data
df = pd.read_excel("data_gp.xlsx")
df = df.drop(['mu*RA', 'mu*t', 'RA/t'], axis=1)
df

,t,mu,RA,XA,XB,QA,Nd,mu/t,VRHE,RA*t,mu/RA
0,0.993000,0.430000,1.3600,1.1000,1.5500,3.00,4.00,0.433000,1.791583,1.350480,0.316176
1,0.998000,0.422000,1.3600,1.1000,1.7300,3.00,5.50,0.423000,1.722750,1.357280,0.310294
2,1.003000,0.415000,1.3600,1.1000,1.9100,3.00,7.00,0.413000,1.707833,1.364080,0.305147
3,0.988000,0.437000,1.3600,1.1000,1.7250,3.00,6.00,0.442000,1.774417,1.343680,0.321324
4,1.004000,0.414000,1.3600,1.1000,1.9020,3.00,6.80,0.413000,1.790833,1.365440,0.304412
5,1.004000,0.413000,1.3600,1.1000,1.8940,3.00,6.60,0.412000,1.753917,1.365440,0.303676
6,1.009000,0.407000,1.3600,1.1000,1.8300,3.00,5.00,0.404000,1.759083,1.372240,0.299265
7,1.010000,0.407000,1.3650,1.1150,1.8300,3.00,5.00,0.403000,1.724667,1.378650,0.298168
8,1.012000,0.407000,1.3700,1.1300,1.8300,3.00,5.00,0.402000,1.755583,1.386440,0.297080
9,1.011000,0.404000,1.3600,1.1000,1.8800,3.00,6.00,0.399000,1.720583,1.374960,0.297059


In [4]:
# Select input features and target
features = ['t', 'mu', 'RA', 'XA', 'XB', 'QA', 'Nd', 'mu/t', 'RA*t', 'mu/RA']
target = 'VRHE'
X = df[features].values
y = df[target].values

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train = X_scaled[:18, :]
y_train = y[:18]
X_val = X_scaled[18:, :]
y_val = y[18:]

X_train, y_train

(array([[-1.08223085,  1.62676953, -0.77345319,  0.86166497, -3.46038823,
          0.97887708, -1.88632043,  1.48595843, -0.89069789,  1.30696623],
        [-0.95938445,  1.18516573, -0.77345319,  0.86166497, -1.30350668,
          0.97887708,  0.04477912,  1.16041762, -0.84320669,  1.07547261],
        [-0.83653805,  0.79876242, -0.77345319,  0.86166497,  0.85337488,
          0.97887708,  1.97587866,  0.83487681, -0.79571548,  0.8729157 ],
        [-1.20507726,  2.01317285, -0.77345319,  0.86166497, -1.36342005,
          0.97887708,  0.68847897,  1.77894515, -0.9381891 ,  1.50952314],
        [-0.81196877,  0.74356194, -0.77345319,  0.86166497,  0.75751347,
          0.97887708,  1.71839873,  0.83487681, -0.78621724,  0.84397899],
        [-0.81196877,  0.68836147, -0.77345319,  0.86166497,  0.66165207,
          0.97887708,  1.46091879,  0.80232273, -0.78621724,  0.81504229],
        [-0.68912236,  0.35715862, -0.77345319,  0.86166497, -0.10523915,
          0.97887708, -0.5989207

Use the same 4-layer model.

In [9]:
class EarlyStoppingWithWarmup(Callback):
    def __init__(self, monitor='val_mae', mode='min', patience=5, warmup_epochs=150, restore_best_weights=True, verbose=1):
        super().__init__()
        self.monitor = monitor
        self.patience = patience
        self.warmup_epochs = warmup_epochs
        self.restore_best_weights = restore_best_weights
        self.verbose = verbose
        
        self.wait = 0
        self.best_weights = None
        self.stopped_epoch = 0
        
        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.inf
        else:
            self.monitor_op = np.greater
            self.best = -np.inf

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current = logs.get(self.monitor)
        if current is None:
            print(f"Warning: EarlyStopping requires {self.monitor} available!")
            return
        
        # Warmup: Skip monitoring during the warmup period
        if epoch < self.warmup_epochs:
            if self.verbose:
                print(f"Epoch {epoch+1}: Warmup phase ({epoch+1}/{self.warmup_epochs}) - {self.monitor}: {current:.4f}")
            return

        if self.monitor_op(current, self.best):
            self.best = current
            self.best_weights = self.model.get_weights()
            self.wait = 0
            if self.verbose:
                print(f"Epoch {epoch+1}: {self.monitor} improved to {current:.4f}")
        else:
            self.wait += 1
            if self.verbose:
                print(f"Epoch {epoch+1}: {self.monitor} did not improve. Wait count: {self.wait}/{self.patience}")
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                if self.restore_best_weights:
                    if self.best_weights is not None:
                        self.model.set_weights(self.best_weights)
                        print(f"Restoring model weights from epoch {self.stopped_epoch - self.patience + 1}")
                print(f"EarlyStopping triggered at epoch {self.stopped_epoch + 1}")

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0 and self.verbose:
            print(f"Training stopped at epoch {self.stopped_epoch + 1} due to early stopping.")


# Setting learning rate decay
initial_lr = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_lr,
    decay_steps=400,
    decay_rate=0.95,
    staircase=True
)

# Create an EarlyStopping callback
early_stop = EarlyStoppingWithWarmup(
    monitor='val_mae',          # Metric to monitor
    patience=5,                 # Wait 5 epochs after min before stopping
    mode='min',                 # Stop when val_mae stops decreasing
    verbose=1,
    restore_best_weights=True
)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.06), input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.06)),
    tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.06)),
    tf.keras.layers.Dense(1)
])

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), loss='mse', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=600, verbose=1, callbacks=[early_stop])

Epoch 1/600
Epoch 1: Warmup phase (1/150) - val_mae: 1.3952 0s 1s/step - loss: 10.0750 - mae: 1.5069
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 10.0750 - mae: 1.5069 - val_loss: 9.6997 - val_mae: 1.3952
Epoch 2/600
Epoch 2: Warmup phase (2/150) - val_mae: 1.3109 0s 23ms/step - loss: 9.7650 - mae: 1.4212
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 9.7650 - mae: 1.4212 - val_loss: 9.4105 - val_mae: 1.3109
Epoch 3/600
Epoch 3: Warmup phase (3/150) - val_mae: 1.2263 0s 22ms/step - loss: 9.4981 - mae: 1.3458
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 9.4981 - mae: 1.3458 - val_loss: 9.1331 - val_mae: 1.2263
Epoch 4/600
Epoch 4: Warmup phase (4/150) - val_mae: 1.1471 0s 26ms/step - loss: 9.2452 - mae: 1.2709
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 9.2452 - mae: 1.2709 - val_loss: 8.8840 - val_mae: 1.1471
Epoch 5/600
Epoch 5: Warmup phase (5/150) - val_mae: 1.0761 0s 22ms/step - loss: 9.0076 - mae: 1.1968
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 9.0076 - mae: 1.1968 - val_los

In [10]:
# Evaluate model
val_loss, val_mae = model.evaluate(X_val, y_val)
print(f"Validation MAE from model.evaluate: {val_mae:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0751 - mae: 0.0210
Validation MAE from model.evaluate: 0.0210


The MAE for training set is 0.0231 eV and validation set is 0.0210 eV.

In [11]:
df_ml = pd.DataFrame({
    'epoch': np.arange(1, len(history.history['loss']) + 1),
    'training loss': history.history['loss'],
    'training mae': history.history['mae'],
    'validation loss': history.history['val_loss'],
    'validation mae': history.history['val_mae']
})

df_ml.to_csv('training_metrics.csv', index=False)

In [13]:
y_pred = model.predict(X_scaled)

df_mlresult = pd.DataFrame({
    'y_target': np.ravel(y),  # Flatten y to 1D too, if needed
    'y_pred': np.ravel(y_pred)
})
df_mlresult

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


,y_target,y_pred
0,1.791583,1.765288
1,1.722750,1.743398
2,1.707833,1.722822
3,1.774417,1.762145
4,1.790833,1.723104
5,1.753917,1.723392
6,1.759083,1.725825
7,1.724667,1.725800
8,1.755583,1.725352
9,1.720583,1.713421


In [14]:
r = permutation_importance(model, X_val, y_val,
                           scoring='r2',
                           n_repeats=30,
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    print(f"{features[i]:<8}"
            f"{r.importances_mean[i]:.3f}"
            f" +/- {r.importances_std[i]:.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━

The new features mu/t and mu/RA make significant contribution to the model performance. Therefore, they cen be added to the feature pool for gplearn analysis to generate more specific mathematical formula to map these features to the OER activities. The result obtained by such neural network study aligns with the results obtained only by gplearn as demonstrated in the paper: Weng, B., Song, Z., Zhu, R. et al. Simple descriptor derived from symbolic regression accelerating the discovery of new perovskite catalysts. Nat Commun 11, 3513 (2020).

In [15]:
r.importances.shape

(10, 30)

In [16]:
df_imp = pd.DataFrame()

for i, feature in enumerate(features):
    df_imp[feature] = r.importances[i]

df_imp

,t,mu,RA,XA,XB,QA,Nd,mu/t,RA*t,mu/RA
0,0.113791,0.006610,0.120206,-0.020125,-0.016782,-0.005156,-0.079315,0.047743,0.103825,0.043477
1,0.117087,0.347818,0.063372,-0.010459,-0.028582,-0.022199,0.175677,0.113774,0.053104,0.102599
2,-0.003665,0.005204,0.001813,-0.055759,0.044438,-0.028347,-0.191259,-0.015435,-0.019450,0.004882
3,0.020899,0.258272,-0.026930,-0.034422,0.018160,-0.033434,-0.028641,0.063193,-0.014125,0.044385
4,0.022605,-0.071532,0.062245,0.084474,0.040626,0.053589,-0.180993,0.004736,0.032192,0.025287
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.117087,0.347818,0.063372,-0.010459,-0.028582,-0.022199,0.175677,0.113774,0.053104,0.102599
7,-0.032882,0.151308,-0.059979,0.078222,0.043598,0.016652,-0.103581,0.045255,-0.048300,0.015323
8,0.129761,0.235697,0.097267,-0.038551,0.035967,-0.043525,-0.042123,0.088222,0.075286,0.100528
9,0.063562,-0.078326,0.102609,-0.007967,0.069732,-0.000634,-0.265688,0.006058,0.069624,0.026787
